## Working with Azure Service Principal and Microsoft Foundry for Production Environments

When deploying applications to production environments using Microsoft Foundry, it's essential to manage access and permissions securely. One common approach is to use an Azure Service Principal (SP) to authenticate and authorize your applications.

### Installing Required Libraries

In [ ]:
%pip install azure-ai-projects==2.0.0b2 openai==1.109.1 python-dotenv azure-identity

### Setting up the Environment Variables

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
load_dotenv()

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")
managed_identity_client_id = os.getenv("MANAGED_IDENTITY_CLIENT_ID")
tenant_id = os.getenv("TENANT_ID")
managed_identity_client_secret = os.getenv("MANAGED_IDENTITY_CLIENT_SECRET")

### Set up the Foundry Project Client

In [ ]:
from azure.identity import ClientSecretCredential

# creating the service principal credential
credential = ClientSecretCredential(
    tenant_id = tenant_id,
    client_id = managed_identity_client_id,
    client_secret = managed_identity_client_secret
)

# creating the Foundry project client
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=credential
)

### Setting Up our Agent

In [ ]:
from azure.ai.projects.models import PromptAgentDefinition

agent_name = "managed-identity-agent"

agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=model_deployment_name,
        instructions="You are a helpful AI assistant",
    ),
)

# printing the agent id
print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.version})")

### Creating a Conversation Object for Agent Chat System

In [ ]:
# creating an openai client first
openai_client = project_client.get_openai_client()

# create a conversation to use with the agent
conversation = openai_client.conversations.create()
print(f"Created conversation with id: {conversation.id}")

### Chat with the Agent

In [ ]:
chat = True

while chat:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        chat = False
        print("Exiting chat. Goodbye!")
    else:
        response = openai_client.responses.create(
            conversation=conversation.id,
            extra_body = {
                "agent": {
                    "name": agent_name,
                    "type": "agent_reference"
                }
            },
            input = user_input
        )

        print(f"Agent: {response.output_text}")